In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import PIL.Image as Image
import cv2

import os
import time
import random
import pathlib

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

In [ ]:
!unzip data.zip

In [4]:
data_dir = pathlib.Path('/content/data')
data_dir

PosixPath('/content/data')

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

5505


In [6]:
#eye - EyeFrame , avi - Aviator, ova - Oval, rec - Rectangle, way - Wayfarer
#sun - Sunglasses
#non - non power reading

images_dict = {
    'eye_avi': list(data_dir.glob('eye_avi/*')),
    'eye_ova': list(data_dir.glob('eye_ova/*')),
    'eye_rec': list(data_dir.glob('eye_rec/*')),
    'eye_way': list(data_dir.glob('eye_way/*')),
    'non_avi': list(data_dir.glob('non_avi/*')),
    'non_ova': list(data_dir.glob('non_ova/*')),
    'non_rec': list(data_dir.glob('non_rec/*')),
    'non_way': list(data_dir.glob('non_way/*')),
    'sun_avi': list(data_dir.glob('sun_avi/*')),
    'sun_ova': list(data_dir.glob('sun_ova/*')),
    'sun_rec': list(data_dir.glob('sun_rec/*')),
    'sun_way': list(data_dir.glob('sun_way/*')),
}
    

In [7]:
images_label_dict = {
    'eye_avi': 0,
    'eye_ova': 1,
    'eye_rec': 2,
    'eye_way': 3,
    'non_avi': 4,
    'non_ova': 5,
    'non_rec': 6,
    'non_way': 7,
    'sun_avi': 8,
    'sun_ova': 9,
    'sun_rec': 10,
    'sun_way': 11,
}


# Create the Model

In [8]:
#Using the pre-trained mobilenetV2 model except the last layer 

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

2021-10-03 12:38:09.567588: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-03 12:38:09.567655: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (745c20f01168): /proc/driver/nvidia/version does not exist


In [9]:
#Adding a layer at the top of the model for classification

classes = 12

model = tf.keras.Sequential([
  pretrained_model ,
  tf.keras.layers.Dense(classes)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 12)                15372     
Total params: 2,273,356
Trainable params: 15,372
Non-trainable params: 2,257,984
_________________________________________________________________


In [10]:
#Reading eyewear images from disk into numpy array using opencv

X, y = [], []

for eyewear, images in images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(images_label_dict[eyewear])
        
X = np.array(X)
y = np.array(y)

# Splitting Training and Testing Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

#Preprocessing the data
X_train_scaled = X_train/255
X_test_scaled = X_test/255

# Train the Model

In [12]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=10)

tcmalloc: large alloc 2485518336 bytes == 0x56505d3f2000 @  0x7f96955ec1e7 0x7f968be2346e 0x7f968be73c7b 0x7f968be73d97 0x7f968be73fe9 0x7f968be76d7d 0x7f9631a8b7d0 0x7f9631a37e30 0x7f9631a38a79 0x564e2bcb20a4 0x564e2bc72c52 0x564e2bce64d9 0x564e2bce09ee 0x564e2bc73bda 0x564e2bce1915 0x564e2bc73afa 0x564e2bce1915 0x564e2bce09ee 0x564e2bc73bda 0x564e2bce2737 0x564e2bce09ee 0x564e2bc73bda 0x564e2bce2737 0x564e2bce09ee 0x564e2bc73bda 0x564e2bce2737 0x564e2bce09ee 0x564e2bbb2e2b 0x564e2bce2fe4 0x564e2bce0ced 0x564e2bc73bda
2021-10-03 12:39:07.809325: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
129/129 [==============================] - 103s 770ms/step - loss: 0.8790 - acc: 0.7229
Epoch 2/10
129/129 [==============================] - 99s 770ms/step - loss: 0.4980 - acc: 0.8314
Epoch 3/10
129/129 [==============================] - 99s 770ms/step - loss: 0.4156 - acc: 0.8600
Epoch 4/10
129/129 [==============================] - 99s 767ms/step - loss: 0.3665 - acc: 0.8760
Epoch 5/10
129/129 [==============================] - 99s 768ms/step - loss: 0.3361 - acc: 0.8883
Epoch 6/10
129/129 [==============================] - 99s 767ms/step - loss: 0.3044 - acc: 0.8970
Epoch 7/10
129/129 [==============================] - 99s 767ms/step - loss: 0.2862 - acc: 0.9007
Epoch 8/10
129/129 [==============================] - 99s 768ms/step - loss: 0.2708 - acc: 0.9087
Epoch 9/10
129/129 [==============================] - 99s 770ms/step - loss: 0.2533 - acc: 0.9167
Epoch 10/10
129/129 [==============================] - 100s 774ms/step - loss: 0.2371 - acc: 0.9268


# Evaluate the Model

In [13]:
model.evaluate(X_test_scaled,y_test)

44/44 [==============================] - 34s 757ms/step - loss: 0.3860 - acc: 0.8758


[0.38597458600997925, 0.8758170008659363]

# Exporting the Model

In [14]:
tf.keras.models.save_model(model,'my_model.hdf5')

In [ ]:
!pip install streamlit

# Deplyoing the Model

In [17]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input

import tensorflow_hub as hub
from sklearn.decomposition import PCA
import scipy

@st.cache(allow_output_mutation=True)
def load_model():
    feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
    KerasLayer = hub.KerasLayer(feature_extractor_model, input_shape=(224, 224, 3), trainable=False)
    model=tf.keras.models.load_model('/content/my_model.hdf5', custom_objects={'KerasLayer': KerasLayer})
    return model

def load_image(path):
    img = image.load_img(path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

def output_images(indexes, images):
    output = []
    for idx in indexes:
        img = cv2.imread(images[idx])
        st.image(img,use_column_width=True )
        output.append(img)
    return output

#using feature extraction to display the similar images
@st.cache(allow_output_mutation=True)
def distance():
    images_path = '/content/data'
    image_extensions = ['.jpg','.png']
    max_num_images = 10000

    images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
    if max_num_images < len(images):
        images = [images[i] for i in sorted(random.sample(xrange(len(images)), max_num_images))]

    #Appending the feature vector 'feat' to the list of features 
    features = []
    for i, image_path in enumerate(images):
        img, x = load_image(image_path);
        feat = model.predict(x)[0]
        features.append(feat)
    
    features = np.array(features)

    #reducing the reduncany in features i.e reducing the number of features in feature vector to 4
    pca = PCA(n_components=4)
    pca.fit(features)
    pca_features = pca.transform(features)
    return pca, pca_features, images


def similar_images(img, pca, pca_features, col_images):
    img = img.resize((224,224))    
    x = np.asarray(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    model = load_model()
    new_feat = model.predict(x)
    new_pca_feat = pca.transform(new_feat)[0]
    
    #using cosine distance as the dissimilarity metric 
    distances = [ scipy.spatial.distance.cosine(new_pca_feat, feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[0:10]  # grab first 10
    results_image = output_images(idx_closest, col_images)


with st.spinner('Model is being loaded..'):
    model=load_model()
    pca, pca_features, col_images = distance()

st.write("""
         # Eyewear Classification
         """
         )


file = st.file_uploader("Please upload the file you want to classify", type=["jpg", "png"])
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_reshape = img[np.newaxis,...]
        prediction = model.predict(img_reshape)
        return prediction

if file is None:
    st.text("Please upload an image file")
else:
    st.write("Uploaded Image")
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    class_names = {0: "EyeFrame-Aviator",1: "EyeFrame-Oval",2: "EyeFrame-Rectangle",3: "EyeFrame-Wayfarer",
                   4: "NPR-Aviator",5: "NPR-Oval",6: "NPR-Rectangle",7: "NPR-Wayfarer",
                   8:"Sunglasses-Aviator",9: "Sunglasses-Oval",10: "Sunglasses-Rectangle",11: "Sunglasses-Wayfarer"}
    st.write(
    "This image most likely belongs to {}"
    .format(class_names[np.argmax(score)])
    )
    pca, pca_features, col_images = distance()
    st.write("Most Similar Images")
    similar_images(image, pca, pca_features, col_images)

Overwriting app.py


In [18]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-10-03 12:56:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  6.12MB/s    in 2.2s    

2021-10-03 12:56:26 (6.12 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [19]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [28]:
get_ipython().system_raw('./ngrok http 8501 &')

In [29]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://0fc2-35-227-30-197.ngrok.io


In [30]:
!streamlit run /content/app.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.227.30.197:8501

2021-10-03 12:58:34.505 Using /tmp/tfhub_modules to cache modules.
2021-10-03 12:58:34.868875: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-03 12:58:34.868941: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (745c20f01168): /proc/driver/nvidia/version does not exist
2021-10-03 12:58:40.130040: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
  Stopping...
